# Problemstellung und Motivation

Frühere Epochen der Großserienfertigung waren durch eine fortschreitende Zentralisierung der Betriebe gekennzeichnet, die auf die Zeit der industriellen Revolution und die Entstehung des Fabriksystems aus der früheren Handwerksproduktion zurückging. Charles Babbage, in “Economy of Machinery and Manufactures” (Babbage 1835), über die Arbeitsökonomie, die durch maschinelle Produktion erleichtert wurde. Die technischen Entwicklungen seiner Zeit wurden begleitet durch die Entstehung von Fabriksystemen und den damit verbundenen Produktivität- und Kostenvorteilen. Zwar können Fabriken effizient produzieren, jedoch ist dieses zentralisierte Paradigma auch durch langwierige, langsam reagierende Lieferketten geprägt. In den letzten drei Jahrzehnten hat Globalisierung die Industrielandschaft mit einzelnen internationalen Produktionsstandorten, die regionale und globale Märkte bedienen, weiter verändert. 

Bei der zentralisierten Werkstattfertigung - also im klassischen Job Shop - wird angenommen, dass es eine einzige Produktionsstätte mit $m$ Maschinen gibt. Das Problem besteht darin $n$ jobs mit jeweils eigenen Prozessrouten so einzutakten, dass eine Zielfunktion minimiert wird. Meist wird hierfür die Fertigungsdauer, als die maximale Zeit zur Fertigstellung aller Jobs, verwendet. Es werden bei den Lösungsverfahren meist folgende Annahmen getroffen:

- Maschinen & Jobs sind kontinuierlich verfügbar
- Rüstzeit können ignoriert werden oder sind in den Prozesszeiten integriert
- Ein Job kann nur auf einer Maschine gleichzeitig bearbeitet werden
- Eine Maschine kann nur einen Job gleichzeitig bearbeiten

Der Trend zu mehreren Fabriken, die die gleichen Güter für unterschiedliche Märkte produzieren, verändert auch die Anforderungen und Problemstellungen der Maschinenbelegungsplanung. Im Distributed Jobs Shop, bestehend aus $f$ Produktionsstätten mit jeweils $m$ Maschinen, wird die Planung komplexer. Es müssen zwei Entscheidungen getroffen werden:
1. Verteilung der Jobs auf die Produktionsstätten 
2. Einplanung der Jobs auf die Maschinen

Im Distributed Jobs Shop Problem werden zusätzlich zum klassischen Job Shop folgende Annahmen getroffen:

- Jobs können nicht mehreren Produktionsstätten bearbeitet werden
- Produktionsstätten haben jeweils einen identischen Maschinenpark

Die Maschinenbelegungsplanung im Distributed Job Shop kann durch die Adaption von bestehenden Regel-basierten Heuristik oder durch Greedy Heuristiken gelöst werden. Die im Paper “Modeling and heuristics for scheduling of distributed job shops” von Nadir B. und Azab A. entwickelten Heuristiken sollen im folgenden näher erläutert werden.

# Anwendungsbeispiel aus der Praxis

Gerade die Halbleiterindustrie lebt schon seit einigen Jahren im Distributed Job Shop innerhalb eines globalen Produktionsnetzwerkes. Zu beobachten ist, dass viele Halbleiterfertiger gerade in der Frontend Produktion (Aufbringen der Transistoren auf dem Wafer) ein weltweites Produktionsnetzwerk haben. In der verschieden Fabriken werden oft auch identische Produkte gefährdet, um den regionalen Bedarf zu sättigen. 

Infineon, beispielsweise, hat 12 Frontend Produktionsstätten weltweit.

![](https://d2mxuefqeaa7sj.cloudfront.net/s_34AC10E35AF26487536528661EC7BDE438B5E36D9B8556471EC64FB0C6D83780_1526112815141_Screen+Shot+2018-05-12+at+10.12.26+AM.png)


Jeder Wafer hat dabei ein spezielles Rezept (Operationsreihenfolge) und wird in dieser Reihenfolge auf unterschiedlichen Maschinen bearbeitet. Selbst die Annahme, dass Rüstzeiten aus der Betrachtung ausgeschlossen werden, trifft bei fast allen Prozessen aufgrund einem hohen Automatisierungsgrad zu.

# Mathematische Problemformulierung

Bei der mathematischen Formulierung des Distributed Job Shops $DJ_m$ wird folgende Notation nach Pinedo verwendet:

- $n$ — Anzahl an der Jobs $j,k = \{0,1,2,\ldots, n\}$, wobei es sich bei job 0 um einen Dummy Job zur Identifizierung des ersten auf einer Maschine zu prozessierenden Jobs handelt
- $m$ — Anzahl der Maschinen $i, l = \{1, 2, \ldots, m\}$
- $f$ — Anzahl der Produktionsstätten $r = \{1, 2, \ldots, f\}$
- $p_{j,i}$ — Porzessierungszeit von Job $j$ auf Maschine $i$


Nach Pinedo sollte das Scheduling Problem durch ein Triplet aus Maschinenumgebung, Prozesscharakteristiken und Zielfunktion beschrieben werden. Im Falle des Distributed Job Shops kann das Problem wie folgt beschrieben werden:

$$J_{f,m} | | C_{max}$$

Im Gegensatz zum klassischen Job Shop Problem, was durch $J_{m} | | C_{max}$ beschrieben wird, muss im Distributed Job Shop die Produktionsstätte $f$ mit in Betracht gezogen werden.

# Setup & Problemerstellung

In [1]:
import pandas as pd
import numpy as np
import math
import random
import timeit
import itertools
from joblib import Parallel, delayed

# Datenset für Vorlesung
problem = pd.read_excel('Job Shop - Simple Example.xlsx')

# Parameter
F = 2
M = 5
J = 10

In [2]:
# Problem Generator
problem = list()
    
for j in range(1, J + 1):

    m_list = ['M' + str(m) for m in range(1, M + 1)]
    random.shuffle(m_list)
    op_list = [op for op in range(1, M + 1)]
    random.shuffle(op_list)

    for m in m_list:

        op = op_list.pop()
        problem.append([j, op, m, np.random.randint(1, 15)])
            
problem = pd.DataFrame(problem, columns=['Job', 'Operation', 'Machine', 'Duration'])

# Regel-basierte Heuristiken
Bei den Regel-basierten Heuristiken wird, wie im ersten Kapitel erklärt, in zwei Schritten vorgegangen. 

## (A) Aufteilung von Jobs auf Produktionsstätten
Bei der Aufteilung der Jobs auf die Produktionsstätte werden folgende Aspekte mit in Betracht gezogen:

- Einbeziehung der Prozesszeiten je Auftrag $j$ auf den verschiedenen Maschinen als Kennzahl für Arbeitspensum
- Möglichst gleichmäßige Aufteilung der Prozesszeiten auf die jeweiligen Maschinen der Produktionsstätten (Ungleichverteilung kann zu schlechter Kapazitätsauslastung führen)
- Jobs mit gleichen Routen sollten auf verschiedene Produktionsstätten verteilt werden, das sie die Fertigungsdauer erhöhen können durch potenziell erhöhte Standzeiten

Durch Naderi et. al. entwickelte Formel für die Kalkulation von Arbeitspensum:
$$\text{Arbeitspensum}(j,i)=\left(\sum_{k \in R_{j,i}} p_{j,k} \right) + p_{j,i} \text{  } \forall_{i j}$$

wobei:

- $R_{j,i}$ — Der Satz aller Maschine $i$ vorgeschalteten Maschinen aus Job $j$

Bei der Verteilung von Jobs auf Fabriken soll das maximale Arbeitspensum je Fabrik und Maschine minimiert werden.

In [5]:
## Darstellung des Problems in aus Vorlesung bekanner Tabellenform
pd.crosstab(problem['Job'], problem['Machine'], problem['Duration'], aggfunc="sum")

Machine,M1,M2,M3,M4,M5
Job,,,,,
1,3,14,10,4,10
2,13,12,7,11,12
3,13,4,11,5,10
4,10,5,1,5,7
5,11,8,14,13,8
6,6,10,11,14,10
7,4,11,7,7,9
8,8,10,1,11,6
9,4,11,10,13,5


In [6]:
# Funktion für die Berechnung des Arbeitspensums
def arbeitspensum(df):
    
    df['Arbeitspensum'] = float('nan')
    columns = df.columns

    result = Parallel(n_jobs=8)(delayed(arbeitspensum_job)(j) for j in df['Job'].unique())
    
    return pd.DataFrame(list(itertools.chain.from_iterable(result)), columns=columns)

In [7]:
def arbeitspensum_job(j, df=problem):
    df = df.loc[(df['Job'] == j), :][:]
    
    for i in df['Machine'].unique():

        # Aktuelle Operation bestimmen
        op = df[df['Machine'] == i]['Operation'].iloc[0]

        # Arbeitspensum berechnen
        load = sum(df[(df['Operation'] <= op)]['Duration'])

        # Arbeitspensum speichern
        df.loc[(df['Machine'] == i) & (df['Operation'] == op), 'Arbeitspensum'] = load
        
    return df.values.tolist()

In [8]:
# Berechnung des Arbeitspensums für das einfache Problem
start = timeit.default_timer()

problem = arbeitspensum(problem)

stop = timeit.default_timer()

print(stop - start)

problem

0.3914162769797258


,Job,Operation,Machine,Duration,Arbeitspensum
0,1,4,M2,14,37.0
1,1,1,M1,3,3.0
2,1,3,M3,10,23.0
3,1,5,M4,4,41.0
4,1,2,M5,10,13.0
5,2,1,M2,12,12.0
6,2,5,M1,13,55.0
7,2,2,M5,12,24.0
8,2,4,M4,11,42.0
9,2,3,M3,7,31.0


In [9]:
start = timeit.default_timer()

# Spalten verwerfen
try:
    problem = problem.drop(['Produktionsstätte'], axis=1)
except:
    pass

try:
    problem = problem.drop(['Arbeitspensum Rank'], axis=1)
except:
    pass

# Reihenfolge der Jobzuordnung bestimmen
load_sum = problem.groupby(['Job'])["Arbeitspensum"].sum().rank(ascending=False, method='first').sort_values()
load_sum = pd.DataFrame({'Job': load_sum.index, 'Arbeitspensum Rank': load_sum.values})
problem = problem.merge(load_sum, on=['Job'])


# Erste Zuordnungsrunde auf Produktionsstätten
for f in range(1, F + 1):
    
    try:
        j = load_sum['Job'].iloc[f - 1]
        problem.loc[problem['Job'] == j, 'Produktionsstätte'] = f
    except:
        break
    
# Zweite Zuordnungsrunde
for j in load_sum['Job']:
    
    # Überspringe bereits zugewiesene jobs
    if sum(problem.loc[problem['Job'] == j, 'Produktionsstätte']) > 0:
        continue
        
    # Minimales maximales Pensum bestimmen
    best_min_load = float("inf")
    best_f = 0
    
    for f in range(1, F + 1):
        problem.loc[problem['Job'] == j, 'Produktionsstätte'] = f
        
        # Minimum
        min_load = pd.DataFrame(problem[problem['Produktionsstätte'] == f].groupby(['Produktionsstätte', 'Machine'])["Arbeitspensum"].sum()).groupby(['Produktionsstätte'])["Arbeitspensum"].max().min()
        if min_load < best_min_load:
            best_min_load = min_load
            best_f = f
    
    problem.loc[problem['Job'] == j, 'Produktionsstätte'] = best_f

stop = timeit.default_timer()

print(stop - start)

display(problem)

0.10915285101509653


,Job,Operation,Machine,Duration,Arbeitspensum,Arbeitspensum Rank,Produktionsstätte
0,1,4,M2,14,37.0,6.0,2.0
1,1,1,M1,3,3.0,6.0,2.0
2,1,3,M3,10,23.0,6.0,2.0
3,1,5,M4,4,41.0,6.0,2.0
4,1,2,M5,10,13.0,6.0,2.0
5,2,1,M2,12,12.0,1.0,1.0
6,2,5,M1,13,55.0,1.0,1.0
7,2,2,M5,12,24.0,1.0,1.0
8,2,4,M4,11,42.0,1.0,1.0
9,2,3,M3,7,31.0,1.0,1.0


In [10]:
# Data Preparation for visualization in Tableau
columns = list(problem.columns)
columns += ['Assignment Step']
export_raw = problem.copy()
export = list()

In [11]:
def tableau_assignments(a, df=export_raw):
    
    if a % 100 == 0:
        print('Currently at Rank ' + str(a))
    
    df = export_raw.copy()
    df['Assignment Step'] = int(a)
    return df[df['Arbeitspensum Rank'] <= a].values.tolist()

In [12]:
print('Preparing Output...')
result = list()

for a in export_raw['Arbeitspensum Rank'].unique():
    result.extend(tableau_assignments(a))

print('Creating Data Frame...')
export = pd.DataFrame(result, columns=columns)
print('Saving Data Frame...')
export.to_csv('zuordnung-' + str(F) + '-' + str(M) + '-' + str(J) + '.csv')
print('Done.')

Preparing Output...
Creating Data Frame...
Saving Data Frame...
Done.


## (B) Einplanung der Aufträge auf Maschinen
Mithilfe der bekannten Heuristiken Shortest Processing Time (SPT) und Longest Processing Time (LPT) werden die den Produktionsstätten zugeordneten Jobs auf den einzelnen Maschinen eingeplant.

### Definition von hilfreichen Funktionen

In [13]:
import os

def clear_solution_file():
    try:
        os.remove('solution-' + str(F) + '-' + str(M) + '-' + str(J) + '.csv')
        print('Removed the existing solution file.')
    except:
        print("Couldn't find and remove an exisiting solution file")
        
def export_solution(df, heuristic):
    
    df['Heuristik'] = heuristic
    
    try:
        existing = pd.read_csv('solution-' + str(F) + '-' + str(M) + '-' + str(J) + '.csv')
        existing = existing[existing['Heuristik'] != heuristic]
        result = existing.append(df)
        cols = [c for c in existing.columns if c.lower()[:4] != 'unna']
        result = result[cols]
        result.to_csv('solution-' + str(F) + '-' + str(M) + '-' + str(J) + '.csv')
        print('Appended solution to existing file.')
    except:
        df.to_csv('solution-' + str(F) + '-' + str(M) + '-' + str(J) + '.csv')
        print('Wrote solution to new file.')

In [14]:
def job_ready(df, j, m, t):
    
    # First operation & not started
    if (df[(df['Job'] == j) & (df['Machine'] == m)]['Operation'].iloc[0] == 1) & (math.isnan(df[(df['Job'] == j) & (df['Machine'] == m)]['Start Time'])):
        return True
    
    # Previous steps performed
    elif df[(df['Job'] == j )]['Start Time'].count() == (df[(df['Job'] == j) & (df['Machine'] == m)]['Operation'].iloc[0] - 1):
        
        # Enough time has passed for job to be finished
        max_start_row = df[df['Job'] == j][df[df['Job'] == j].index == df[df['Job'] == j]['Start Time'].idxmax()]

        if (max_start_row['Start Time'] + max_start_row['Duration']).values[0] <= t:
            return True
    
    return False

In [15]:
def machine_ready(df, m, t):

    # Not yet used
    if df[df['Machine'] == m]['Start Time'].count() == 0:
        return True

    # Previous Jobs finished
    max_start_row = df[df['Machine'] == m][df[df['Machine'] == m].index == df[df['Machine'] == m]['Start Time'].idxmax()]

    if (max_start_row['Start Time'] + max_start_row['Duration']).values[0] <= t:
        return True
    
    return False

In [16]:
def schedule_in_order(df):
    
    df['Start Time'] = float('nan')
    
    # Sequence according to rules
    for f in df['Produktionsstätte'].unique():

        t = 0
        t_next = True

        # Continue until all Operations have a start time
        while t_next:

            for m in df[(df['Produktionsstätte'] == f)]['Machine'].unique():
                if not machine_ready(df[df['Produktionsstätte'] == f], m, t):
                    continue

                for j in df[(df['Produktionsstätte'] == f) & (df['Machine'] == m)]['Job']:
                    if job_ready(df[df['Produktionsstätte'] == f], j, m, t):
                        df.loc[(df['Produktionsstätte'] == f) & (df['Machine'] == m) & (df['Job'] == j), ['Start Time']] = t
                        break

            if df[(df['Produktionsstätte'] == f)]['Start Time'].count() == len(df[(df['Produktionsstätte'] == f)]['Start Time']):
                t_next = False
                break

            t += 1
            
    df['Finish Time'] = df['Start Time'] + df['Duration']
    
    return df

In [17]:
clear_solution_file()

Couldn't find and remove an exisiting solution file


### Shortest Processing Time
Folgende Tabelle erstellen und für jede Maschine nach der Prozesszeit aufsteigend sortieren.

| Machine | Job | Operation | Prozesszeit | Startzeit | Endzeit |
| ------- | --- | --------- | ----------- | --------- | ------- |
| M1      | 1   | 3         | 3           |           |         |
| M1      | 3   | 1         | 12          |           |         |
| M2      | 3   | 3         | 1           |           |         |
| M2      | 1   | 2         | 7           |           |         |
| M3      | 1   | 1         | 4           |           |         |
| M3      | 3   | 2         | 6           |           |         |

#### Iteration 1 … x
Jeweils die erste Zeile pro Maschine betrachten. Alle vorhergehenden Operationen abgeschlossen?

1. Ja? max( Maximale Endzeit der Maschine, Maximale Endzeit des Jobs ) = neue Startzeit. Neue Endzeit ergibt sich aus der neuen Startzeit + Prozesszeit. Fortfahren mit nächster Maschine.
2. Nein? Fortfahren mit nächster Zeile der Maschine.

| Machine | Job | Operation | Prozesszeit | Startzeit | Endzeit |
| ------- | --- | --------- | ----------- | --------- | ------- |
| M1      | 1   | 3         | 3           | x         |         |
| M1      | 3   | 1         | 12          |           |         |
| M2      | 3   | 3         | 1           | x         |         |
| M2      | 1   | 2         | 7           |           |         |
| M3      | 1   | 1         | 4           | 0         | 4       |
| M3      | 3   | 2         | 6           |           |         |

Nach Fertigstellung:

| Machine | Job | Operation | Prozesszeit | Startzeit | Endzeit |
| ------- | --- | --------- | ----------- | --------- | ------- |
| M1      | 1   | 3         | 3           | 11        | 14      |
| M1      | 3   | 1         | 12          | 0         | 12      |
| M2      | 3   | 3         | 1           | 18        | 19      |
| M2      | 1   | 2         | 7           | 4         | 11      |
| M3      | 1   | 1         | 4           | 0         | 4       |
| M3      | 3   | 2         | 6           | 12        | 18      |

In der Präsentation wurde zu Illustrationszwecken unter anderem folgendes interaktives Beispiel verwendet:

In [18]:
def sequence_spt(df):
    
    # Sort by shortest processing time
    df = df.sort_values(by=['Produktionsstätte', 'Machine', 'Duration'], ascending=True)
    
    # Sequence according to rules
    df = schedule_in_order(df)
            
    display(df)
    return df

In [19]:
solution_spt = sequence_spt(problem)
export_solution(solution_spt, 'spt')

,Job,Operation,Machine,Duration,Arbeitspensum,Arbeitspensum Rank,Produktionsstätte,Start Time,Finish Time
32,7,1,M1,4,4.0,9.0,1.0,0.0,4.0
41,9,1,M1,4,4.0,5.0,1.0,4.0,8.0
36,8,1,M1,8,8.0,7.0,1.0,8.0,16.0
6,2,5,M1,13,55.0,1.0,1.0,52.0,65.0
12,3,2,M1,13,24.0,4.0,1.0,16.0,29.0
14,3,4,M2,4,33.0,4.0,1.0,37.0,41.0
39,8,2,M2,10,18.0,7.0,1.0,16.0,26.0
34,7,5,M2,11,38.0,9.0,1.0,41.0,52.0
40,9,3,M2,11,28.0,5.0,1.0,26.0,37.0
5,2,1,M2,12,12.0,1.0,1.0,0.0,12.0


Wrote solution to new file.


### Longest Processing Time
Die Schritte für LPT sind identisch zu den von SPT mit der Ausnahme, dass die Tabelle zu Beginn nicht aufsteigend, sondern absteigend sortiert wird.

In [20]:
def sequence_lpt(df):
    
    # Sort by shortest processing time
    df = df.sort_values(by=['Produktionsstätte', 'Machine', 'Duration'], ascending=False)
    
    # Sequence according to rules
    df = schedule_in_order(df)
            
    display(df)
    return df

In [21]:
solution_lpt = sequence_lpt(problem)
export_solution(solution_lpt, 'lpt')

,Job,Operation,Machine,Duration,Arbeitspensum,Arbeitspensum Rank,Produktionsstätte,Start Time,Finish Time
4,1,2,M5,10,13.0,6.0,2.0,15.0,25.0
25,6,3,M5,10,34.0,2.0,2.0,29.0,39.0
22,5,1,M5,8,8.0,3.0,2.0,0.0,8.0
16,4,2,M5,7,12.0,10.0,2.0,8.0,15.0
49,10,2,M5,4,14.0,8.0,2.0,25.0,29.0
28,6,1,M4,14,14.0,2.0,2.0,0.0,14.0
24,5,5,M4,13,54.0,3.0,2.0,41.0,54.0
15,4,5,M4,5,28.0,10.0,2.0,54.0,59.0
3,1,5,M4,4,41.0,6.0,2.0,59.0,63.0
47,10,4,M4,4,24.0,8.0,2.0,63.0,67.0


Appended solution to existing file.


# Greedy-Heuristiken
Im “Modeling and heuristics for scheduling of distributed job shops” Paper von Nadir B. und Azab A. werden weiterhin drei hoch-performante Greedy-Heuristiken eingeführt. Diese Heuristiken beruhen auf dem “insertion neighborhood concept”. D.h. es wird durch kontinuierliches Einsetzen eines Prozessschrittes an verschiedenen Stellen in der Prozesskette die Reihenfolge ermittelt, welche zur kürzesten Fertigungsdauer führt.

## Defintion von hilfreichen Funktionen

In [22]:
# Überprüft, ob eine neue Sequenz im Vergleich zur vorherigen redundant ist
def seq_redundancy(lookup_df, seq_check, seq):
    
    seq_check = np.array(seq_check)
    seq = np.array(seq)

    # Subtract the frames to catch the differences
    diff = np.abs(np.subtract(seq_check, seq))

    # If all 0 then redundant, because belong to same job
    if sum(diff) == 0:
        return True

    # Are the different jobs processed on the same machine?
    mask = np.abs(np.subtract(seq_check, seq)) > 0
    idxs = np.array(range(0, len(seq)))[mask]

    # For each job determine the machine by looking at the operation
    machines = list()
    for idx in idxs:
        j = seq[idx]
        op = sum(seq[:idx] == j) + 1
        
        machines.append(lookup_df[(lookup_df['Job'] == j) & (lookup_df['Operation'] == op)]['Machine'].iloc[0])
    
    if machines[0] == machines[1]:
        return True

    return False

In [23]:
def greedy_schedule(lookup_df, seq):
    seq = np.array(seq)

    df_order = list()
    for idx in range(0, len(seq)):

        j = seq[idx]
        op = sum(seq[:idx] == j) + 1

        df_order.extend(lookup_df[(lookup_df['Job'] == j) & (lookup_df['Operation'] == op)].index.values)

    df = lookup_df.loc[df_order]
    df['Finish Time'] = 0

    for index, row in df.iterrows():
        machine = row['Machine']
        
        # Get Max of Finish time for Machine and Job
        start_m = max(df[df['Machine'] == machine]['Finish Time'])
        start_j = max(df[df['Job'] == row['Job']]['Finish Time'])
        start = max(start_m, start_j)
        
        finish = start + row['Duration']
        df.loc[index, 'Start Time'] = start
        df.loc[index, 'Finish Time'] = finish
        
    return df

## Greedy-Heuristik 1

*Schritt 1.* Zuerst wird wie im vorherigen Abschnitt eingeführt, die Zuteilung von Jobs auf die einzelnen Produktionsstätten mithilfe der Kenngröße des Arbeitspensums durchgeführt.

*Schritt 2.* Für jede Produktionsstätte wird eine zufällige Reihenfolge der Operationen (Operationsfolge) bestimmt, wobei eine spezielle Array-Datenstruktur verwendet wird. Anstatt sowohl Job, Machine und Operationsnummer zu verwenden, wird nur die Jobnummer verwendet welche $m$ Mal wiederholt wird. Durch die Position und die Anzahl der Vorkommnisse der Jobnummer vorher, kann wieder auf Maschine und Operationsnummer zurückgeschlossen werden. Produktionsstätte 1, aus dem vorher vorgestelltem Beispiel, wurde die Jobs $(1, 3)$ zugewiesen, wobei jeder der Jobs auf drei Maschinen bearbeitet wird. Somit ergibt sich, dass beide Jobnummer jeweils drei Mal vorkommen müssen. Die zufällige generierte Reihenfolge der Operationen könnte daher z.B. $[1,3,3,1,3,3]$ lauten. Das würde bedeuten, dass Job $1$ auf Maschine M3 bearbeitet wird, bevor mit der Bearbeitung von Job $3$ auf Maschine M1 begonnen wird.

*Schritt 3.* Für jede Produktionsstätte wird der Länge des Arrays entsprechend viele Iterationen durchgeführt. Der Laufindex `i` wird für die Iterationszahl eingeführt und es wird die Variable `best_seq` verwendet, in der die beste ermittelte Operationsfolge gespeichert wird. Die Jobindizes werden nacheinander aus der ursprünglichen Operationsfolge genommen und in alle möglichen Positionen in der Operationsfolge getestet. Schließlich wird die beste Position ausgewählt. Die Prozedur wird für den nächsten Job wiederholt.

In [24]:
start = timeit.default_timer()

loops = 0
red_seqs = 0

colums = list()
final_schedule = list()
solution_seq_gh1 = dict()

for k in range(1, F + 1):
    
    lookup_df = problem[problem['Produktionsstätte'] == k]
    
    # Determine random initial order of ops assing to facility k
    ops = problem[problem['Produktionsstätte'] == k]['Job'].values.tolist()
    random.shuffle(ops)
    
    best_seq = None
    best_schedule = None
    
    # We have jobs * operations iterations to go through for each facility
    for i in range(1, len(ops)):
        
        # Reset best finish since for each iteration the best finish will
        # be larger than in the previous one because we are adding ops
        best_finish = float('inf')
        
        # Keep track of the last sequence that was checked
        seq_check = None
        
        # Take ith job number from the initial order
        op = ops[i]
        
        # If no best_seq exists yet use a random one
        if best_seq == None:
            i_seq = ops[:i]
        else:
            i_seq = best_seq
        
        # Try each insertion point
        for h in range(0, (i + 1)):
            
            # stats
            loops += 1
            
            # Sart with the standard sequence of the iteration
            seq = i_seq[:]
            
            # Insert into the list position
            seq.insert(h, op)
            
            # Check redundancy
            '''if seq_check != None:
                if seq_redundancy(lookup_df, seq_check, seq):
                    red_seqs += 1
                    continue
            '''
            # Save the current sequence for the next red check
            seq_check = seq[:]
            
            # Test inserting the job, schedule and see if best
            schedule = greedy_schedule(lookup_df, seq)
            finish = max(schedule['Finish Time'])
            
            if finish < best_finish:
                best_seq = seq
                best_finish = finish
                best_schedule = schedule
    
    # Keep track of the best schedule for each facility
    columns = list(best_schedule.columns)
    final_schedule.extend(best_schedule.values.tolist())
    solution_seq_gh1[k] = best_seq
    
solution_gh1 = pd.DataFrame(final_schedule, columns=columns)

stop = timeit.default_timer()

print(stop - start)

export_solution(solution_gh1, 'gh1')
display(solution_gh1)

32.3692359419947
Appended solution to existing file.


,Job,Operation,Machine,Duration,Arbeitspensum,Arbeitspensum Rank,Produktionsstätte,Finish Time,Start Time,Heuristik
0,3,1,M3,11,11.0,4.0,1.0,11,0.0,gh1
1,8,1,M1,8,8.0,7.0,1.0,8,0.0,gh1
2,7,1,M1,4,4.0,9.0,1.0,12,8.0,gh1
3,2,1,M2,12,12.0,1.0,1.0,12,0.0,gh1
4,8,2,M2,10,18.0,7.0,1.0,22,12.0,gh1
5,2,2,M5,12,24.0,1.0,1.0,24,12.0,gh1
6,7,2,M3,7,11.0,9.0,1.0,19,12.0,gh1
7,9,1,M1,4,4.0,5.0,1.0,16,12.0,gh1
8,3,2,M1,13,24.0,4.0,1.0,29,16.0,gh1
9,9,2,M4,13,17.0,5.0,1.0,29,16.0,gh1


## Greedy Heuristik 2

Die zweite Heuristik funktioniert ähnlich der ersten Heuristik. Der einzige Unterschied ist die Zuordnung der Jobs zu den Produktionsstätten.

*Schritt 1.* Zuerst wird die Liste der Jobs absteigend nach der Fertigungsdauer sortiert. Die ersten $f$ Jobs werden jeweils einer Produktionsstätte zugeordnet.

*Schritt 2.* Die restlichen Jobs werden weiter in der absteigenden Reihenfolge bearbeitet und jeweils der Produktionsstätte mit der niedrigsten gesamten Fertigungsdauer zugeordnet. Es wird nun der dritte Schritt aus der ersten Greedy-Hseuristik angewendet.

In [25]:
start = timeit.default_timer()

df = problem[:]
df['Produktionsstätte'] = float('nan')

# Sort Jobs by their total processing times
jobs_ordered = problem.groupby(['Job'])["Duration"].sum().sort_values(ascending=False).index.tolist()

best_seq = dict()
best_finish = list()
best_schedule = dict()

# Assigning the first f jobs to different facilities
for f in range(1, (F + 1)):

    j = jobs_ordered.pop(0)
    
    best_seq[f] = [j, j, j]

    df.loc[df['Job'] == j, 'Produktionsstätte'] = f
    
    # Calculate Makespan
    schedule = greedy_schedule(df[df['Produktionsstätte'] == f], best_seq[f])
    makespan = max(schedule['Finish Time'])
    
    best_finish.append(makespan)
    
for j in jobs_ordered:
    
    # Select the factory with the lowest makespan
    f = (min(range(len(best_finish)), key=best_finish.__getitem__) + 1)
    
    # print('Factory: {}, Array {}'.format(f, best_finish))
    
    df.loc[df['Job'] == j, 'Produktionsstätte'] = f
    
    # Try each insertion point for each operation
    for i in range(3):
        
        # print('Iteration {}'.format(i))
        
        # Keep track of the last sequence that was checked
        seq_check = None
        
        ref_finish = float('inf')
        ref_seq = best_seq[f][:]
        
        for h in range(0, (len(best_seq[f]) + 1)):
            

            # Sart with the standard sequence of the iteration
            seq = ref_seq[:]

            # Insert into the list position
            seq.insert(h, j)

            # Check redundancy
            '''
            if seq_check != None:
                if seq_redundancy(df[df['Produktionsstätte'] == f], seq_check, seq):
                    continue
            '''
            # Save the current sequence for the next red check
            seq_check = seq[:]

            # Test inserting the job, schedule and see if best
            schedule = greedy_schedule(df[df['Produktionsstätte'] == f], seq)
            finish = max(schedule['Finish Time'])
            
            # print('Ref: {}, Seq: {}, Fin: {}'.format(ref_seq, seq, finish))
            # print('Ref Finish: {}'.format(ref_finish))

            if finish < ref_finish:
                ref_finish = finish
                best_seq[f] = seq
                best_finish[int(f - 1)] = finish
                best_schedule[f] = schedule

final_schedule = list()
columns = list(best_schedule[1].columns)

for key in best_schedule:
    final_schedule.extend(best_schedule[key].values.tolist())
    
solution_gh2 = pd.DataFrame(final_schedule, columns=columns)

stop = timeit.default_timer()

print(stop - start)

export_solution(solution_gh2, 'gh2')
display(solution_gh2)

8.1062264170032
Appended solution to existing file.


,Job,Operation,Machine,Duration,Arbeitspensum,Arbeitspensum Rank,Produktionsstätte,Finish Time,Start Time,Heuristik
0,4,1,M2,5,5.0,10.0,2.0,5,0.0,gh2
1,10,1,M1,10,10.0,8.0,2.0,10,0.0,gh2
2,8,1,M1,8,8.0,7.0,2.0,18,10.0,gh2
3,6,1,M4,14,14.0,2.0,2.0,14,0.0,gh2
4,5,1,M5,8,8.0,3.0,2.0,8,0.0,gh2
5,10,2,M5,4,14.0,8.0,2.0,14,10.0,gh2
6,4,2,M5,7,12.0,10.0,2.0,21,14.0,gh2
7,4,3,M1,10,22.0,10.0,2.0,31,21.0,gh2
8,10,3,M3,6,20.0,8.0,2.0,20,14.0,gh2
9,5,2,M2,8,16.0,3.0,2.0,16,8.0,gh2


## Greedy Heuristik 3
Die dritte Heuristik funktioniert ebenfalls ähnlich der ersten Heuristik. Der Unterschied ist, dass alle Permutationen inklusive die Zuordnung zu allen Produktionsstätten in jeder Iteration getestet wird.

*Schritt 1.* Zuerst wird die Liste der Jobs erstellt. Dabei erfolgt keine Sortierung.

*Schritt 2.* Für jede Kombination aus Job und Fabrik wird der dritte Schritt aus der ersten Greedy Heuristik angewendet. Der Job wird der Fabrik zugeordnet, dessen Fertigungsdauer am Ende minimal ist.

In [26]:
start = timeit.default_timer()

df = problem[:]
df['Produktionsstätte'] = float('nan')

# Create job list in random order
jobs_ordered = list(problem['Job'].unique())
jobs_ordered

best_seq = dict()
best_schedule = dict()

# Assigning the first f jobs to different facilities
for f in range(1, (F + 1)):

    j = jobs_ordered.pop(0)
    best_seq[f] = [j, j, j]

# Handle all remaining jobs
for j in jobs_ordered:
    
    # Keep track of the best stats for the factories
    fbest_f = None
    fbest_seq = None
    fbest_finish = float('inf')
    fbest_schedule = None
    
    # Search each factory
    for f in range(1, (F + 1)):
        
        # print('Factory {}'.format(f))
        
        # Pretend factory assignment
        df['Produktionsstätte'] = f
    
        # Try each insertion point for each operation
        for i in range(3):
            
            if i == 0:
                # Get the best sequence developed so far for the factory
                iref_seq = best_seq[f][:]
            
            # print('Iteration {}'.format(i))

            # Reset the last sequence that was checked
            seq_check = None

            ibest_seq = None
            ibest_finish = float('inf')
            ibest_schedule = None

            for h in range(0, (len(iref_seq) + 1)):

                # Sart with the standard sequence of the iteration
                seq = iref_seq[:]

                # Insert into the list position
                seq.insert(h, j)

                # Check redundancy
                '''if seq_check != None:
                    if seq_redundancy(df[df['Produktionsstätte'] == f], seq_check, seq):
                        # print('Red: {}'.format(seq))
                        continue
                '''
                # Save the current sequence for the next red check
                seq_check = seq[:]

                # Test inserting the job, schedule and see if best
                schedule = greedy_schedule(df[df['Produktionsstätte'] == f], seq)
                finish = max(schedule['Finish Time'])
                # print('Ref: {}, Seq: {}, Fin: {}'.format(iref_seq, seq, finish))
            
                # Get the best sequence within the factory
                if finish < ibest_finish:
                    ibest_seq = seq
                    ibest_finish = finish
                    ibest_schedule = schedule
            
            iref_seq = ibest_seq[:]
            # print('New Reference: {}'.format(ibest_seq[:]))
            # print('{}, best {}'.format(i, ibest_finish))
            
        # Determine the best sequence overall & assign factory
        if ibest_finish < fbest_finish:
            # print('New Factory Best old {} - new {}'.format(fbest_finish, ibest_finish))
            fbest_f = f
            fbest_seq = ibest_seq
            fbest_finish = ibest_finish
            fbest_schedule = ibest_schedule
              
    best_seq[fbest_f] = fbest_seq
    # print(best_seq)
    best_schedule[fbest_f] = fbest_schedule
    
final_schedule = list()
columns = list(best_schedule[1].columns)

for key in best_schedule:
    final_schedule.extend(best_schedule[key].values.tolist())
    
solution_gh3 = pd.DataFrame(final_schedule, columns=columns)

stop = timeit.default_timer()

print(stop - start)

display(solution_gh3)
export_solution(solution_gh3, 'gh3')

18.55089350300841


,Job,Operation,Machine,Duration,Arbeitspensum,Arbeitspensum Rank,Produktionsstätte,Finish Time,Start Time
0,9,1,M1,4,4.0,5.0,1,4,0.0
1,9,2,M4,13,17.0,5.0,1,17,4.0
2,7,1,M1,4,4.0,9.0,1,8,4.0
3,5,1,M5,8,8.0,3.0,1,8,0.0
4,4,1,M2,5,5.0,10.0,1,5,0.0
5,5,2,M2,8,16.0,3.0,1,16,8.0
6,9,3,M2,11,28.0,5.0,1,28,17.0
7,4,2,M5,7,12.0,10.0,1,15,8.0
8,3,1,M3,11,11.0,4.0,1,11,0.0
9,7,2,M3,7,11.0,9.0,1,18,11.0


Appended solution to existing file.


Auswertung und Analyse der Ergebnisse wurde in einem Tableau Workbook durchgeführt.